In [1]:
import pandas as pd

In [2]:
df_oh = pd.read_pickle('./data_frames/dense_acs_mm_oh.pkl')

In [3]:
df_oh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1569188 entries, 0 to 1630166
Columns: 898 entries, WKHP to RAC1P_9
dtypes: float64(3), uint8(895)
memory usage: 1.4 GB


In [4]:
df_oh.head()

,WKHP,AGEP,PINCP,OCCP_10,OCCP_20,OCCP_40,OCCP_51,OCCP_52,OCCP_60,OCCP_101,...,MAR_5,RAC1P_1,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9
0,0.295918,0.025316,0.039472,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,0.397959,0.037975,0.030967,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,0.173469,0.025316,0.030467,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0.051020,0.215190,0.053479,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0.091837,0.025316,0.009455,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [5]:
from build_all_models import build_baseline_alwayson

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_oh.drop('PINCP', axis=1), 
                                                    df_oh['PINCP'], test_size = 0.2,
                                                    shuffle=False)

In [17]:
X_dat = df_oh.drop('PINCP', axis=1)
y_dat = df_oh['PINCP']

In [18]:
import numpy as np
A_t = np.load('./data_frames/nparrays/allstates2021_20230724-233202_unshuffled__dropped_.npy')

In [9]:
# np.all(A_t1 == A_t2)

In [19]:
np.all(df_oh['RAC1P_2'] == A_t[:, 3])

True

In [31]:
dirname = 'onehot_encoded'
filename = 'linearmodelLS'

In [12]:
A_t[:100].shape

(100, 12)

In [33]:
from OnlineLinModel_alwaysactive_implementable import *
expert = OnlineLinearModel_alwaysactive(l2_pen=0.0)

In [34]:
X_train.dtypes

WKHP       float64
AGEP       float64
OCCP_10      uint8
OCCP_20      uint8
OCCP_40      uint8
            ...   
RAC1P_5      uint8
RAC1P_6      uint8
RAC1P_7      uint8
RAC1P_8      uint8
RAC1P_9      uint8
Length: 897, dtype: object

In [35]:
X_dat.shape, y_dat.shape, A_t.shape

((1569188, 897), (1569188,), (1569188, 12))

In [ ]:
# bbass = build_baseline_alwayson(dirname, filename, X_train.iloc[:1000], \
#                                   y_train.iloc[:1000], A_t[:1000], expert, l2_pen=0.0)

bbass = build_baseline_alwayson(dirname, filename, X_dat, y_dat, A_t, expert, l2_pen=0.0)

  5%|▌         | 78876/1569188 [01:41<31:34, 786.63it/s]

In [ ]:
for gnum in range(A_t.shape[1]):
    print(f'''group num {gnum}, {bbass.implementable_expert.cumloss_groupwise_oridge[gnum][-1]} ''')

In [16]:
import joblib
olinear = joblib.load('./onehot_encoded/models/linear_sepbuild/test_new20230820-220448_l2pen=0.0.pkl')

FileNotFoundError: [Errno 2] No such file or directory: './onehot_encoded/models/linear_sepbuild/test_new20230820-220448_l2pen=0.0.pkl'

In [ ]:
olinear.cumloss_groupwise_oridge.shape